# Exercise 1: Consumption-leisure choice

In the lecture we analyzed how a household optimally chooses consumption and labor supply. In this exercise, we revisit this setting, imposing different assumptions on the household's utility function.

Assume that the household chooses *consumption* $c$ and *leisure* $\ell$ in order to maximize
$$
\begin{aligned}
\max_{c,~\ell} ~\Bigl\{\log(c) &+ \psi \log(\ell) \Bigr\} \\ 
\text{subject to} \enskip c &= a + (1-\ell)w \\
    c &\geq 0 \\
    0 &\leq \ell \leq 1
\end{aligned}
$$
The second line represents the budget constraint, where $a$ is the level of assets and $w$ is the wage rate. The last line imposes that leisure has to be between 0 and 1, where 1 represents the entire time endowment of the household. Consequently, for a given leisure choice, the household supplies the remaining time $1-\ell$ in the labor market and earns $(1-\ell)w$ in labor income.

***
# Exercise 2: Consumption & labor supply with commuting

Recall the consumption & labor supply problem we studied in the lecture:
the individual has preferences over consumption and leisure given by
$$
u(c,h) = \frac{c^{1-\gamma} - 1}{1-\gamma} - \psi \frac{h^{1+1/\theta}}{1+1/\theta}
$$
where $c \geq 0$ is the amount consumed, and $h \geq 0$ (for "hours") is the amount of labor the individual wishes to work (the remaining time is then consumed as leisure). The individual does not like working, and therefore the amount of hours worked enters as a disutility term in the utility function.
Note that for the special case of $\gamma = 1$, the utility from consumption becomes $\log(c)$,
the natural logarithm of $c$, so that
$$
u(c,h) = \log(c) - \psi \frac{h^{1+1/\theta}}{1+1/\theta}
$$


Unlike in the lecture, we assume that the individual can either work part-time for $h < H$ hours in their neighborhood for a lower wage $w_{\ell}$ 
or commute to the city to work full-time for $h \geq H$ hours at a higher wage $w_h$. 
However, commuting to the city incurs a fixed commuting cost $\kappa$. 
The budget constraint is therefore given by 
$$
c = \begin{cases}
a + w_{\ell} \cdot h & \text{if } h < H \\
a + w_{h} \cdot h - \kappa & \text{if } h \geq H \\
\end{cases}
$$
This is an example of a problem that is harder to solve analytically because of the discountinuous jump at the choice $h=H$.
As you will see, this discontinuity can also cause problems for numerical solvers.

To get you started, use the following function to evaluate the utility for given choices and parameters (it's the same one we wrote in the lecture):

In [ ]:
import numpy as np

def util(c, h, gamma, psi, theta):
    """
    Compute the utility of a given consumption/labor supply choice.

    Parameters
    ----------
    c : float or array
        Consumption level.
    h : float or array
        Hours worked.
    gamma : float
        Relative risk aversion parameter.
    psi : float
        Weight on disutility of labor.
    theta : float
        Labor supply elasticity.

    Returns
    -------
    u : float or array
        Utility value.
    """

    # Consumption utility
    if gamma == 1:
        # Log utility
        u = np.log(c)
    else:
        # General CRRA utility
        u = (c**(1-gamma) - 1) / (1-gamma)

    # add disutility of labor
    u -= psi * h**(1 + 1/theta) / (1 + 1/theta)

    return u

In the remainder of this exercise, you're asked to find the optimal choices using grid search and two different SciPy minimizer.
Use the following parameters for these tasks:

In [ ]:
# Parameters
a = 0           # initial assets
w_l = 0.75      # low wage (neighborhood)
w_h = 3         # high wage (city)
gamma = 1.0     # Relative risk aversion
psi = 2.0       # weight on disutility of labor
theta = 0.5     # labor supply elasticity
H = 1.0         # cut-off for part-time work
kappa = 2.0     # commuting cost

## Tasks

1.  Write a function with the signature

    ```python
    def util_h(h, gamma, psi, theta, H, kappa, a, w_l, w_h):
        """
        Compute utility for given labor choice and parameters.
        """
    ```

    which uses `util()` to return the utility associated with a hours choice `h`.
    Use the budget constraint to obtained the implied consumption level.

    *Hint:* To get a vectorized function (which can deal with `h` arguments that are arrays),
    use 
    [`np.where()`](https://numpy.org/doc/stable/reference/generated/numpy.where.html)
    instead of an `if` statement when dealing with the two cases in
    the budget constraint.

2.  Solve the problem using grid search, evaluating utility
    on a grid of 10,000 points for $h$ on the interval $[0.1, 2]$.
    Plot utility as a function of $h$ for the values on the hours grid.

3.  Use the scalar minimizer
    [`minimize_scalar()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html)
    and the function `util_h()` you wrote
    to locate the optimum. Do you get a similar result as with grid search?

4.  Lastly, use `util_h()` and a derivative-based minimizer implemented by 
    [`minimize()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)
    to find the optimum.

    *Hint:* Call this minimizer as follows:

    ```python
    res = minimize(
        lambda x: -util_h(x, gamma, psi, theta, H, kappa, a, w_l, w_h), 
        x0 = x0,
        method='L-BFGS-B',
        bounds=((0, None),)
    )
    ```

    This selects the `'L-BFGS-B'` algorithm, a derivative-based method
    which also supports bounds. We impose these using the 
    `bounds=((0, None),)` argument which imposes the $h \geq 0$ lower bound.

    You need to specify an initial guess `x0`: experiment with `x0=0.75`
    and `x0=1.1`. Do these yield the same result?

    *Hint:* The `minimize()` returns a result object where the optimum 
    is stored in the `x` attribute, which in this case is an array of length 1.


***
# Exercise 3: Consumption-savings with uncertain income

We have studied the two-period consumption savings problems several times now, but so far, income in the second period was deterministic.
This setting can be extended to uncertain income, a feature that is at the core of modern macroeconomics.

Consider a household which maximizes *expected* lifetime utility by choosing period-1 consumption $c_1$ and savings $s$,
$$
\begin{aligned}
\max_{c_1,~s} \enskip & \Bigl\{ u(c_1) + \beta \mathbb{E}\bigl[u(c_2)\bigr] \Bigr\} \\
\text{s.t.} \quad c_1 + s &= a \\
                  c_2 &= (1+r)s + y_2 \\
    c_1 &\geq 0, ~ s \geq 0
\end{aligned}
$$
Per-period utility $u(c)$ is the CRRA utility function given by
$$
u(c) = \begin{cases}
    \frac{c^{1-\gamma}}{1-\gamma} & \text{if } \gamma \neq 1 \\
    \log(c) & \text{if } \gamma = 1
    \end{cases}
$$
where $\gamma$ is the RRA coefficient and $\log(\bullet)$ denotes
the natural logarithm.

The savings $s$ generate a gross return $(1+r)$ in the second period. Additionally, the household receives stochastic income $y_2$.
For simplicity, we assume that $y_2$ fluctuates symmetrically around its mean and can take on only two realizations:
$$
y_2 = \begin{cases}
\overline{y} + \epsilon & \text{with probability } \frac{1}{2} \\
\overline{y} - \epsilon & \text{with probability } \frac{1}{2} \\
\end{cases}
$$
You can easily verify that $\mathbb{E}[y_2] = \overline{y}$ and $\text{Var}(y_2) = \epsilon^2$ so that the parameters $\overline{y}$ and $\epsilon$
govern the mean and the volatility of period-2 income, respectively.

## Analytical results

In general, it is not possible to solve such a problem analytically unless we are willing to impose simplifying assumptions (e.g., setting $\gamma=1$,
but even then the optimal $s^*$ is characterized by a tedious quadratic equation).

Nevertheless, we can use the Euler equation for this problem,
$$
u'(c_1) = \beta (1+r) \mathbb{E} \bigl[ u'(c_2) \bigr]
$$
to solve the problem numerically using root-finding. For this purpose, we use the functional form assumption for $u(\bullet)$ and plug in the period-specific budget constraints into the Euler equation to obtain,
<a id="E3:EE"></a>
$$
\bigl(a -s \bigr)^{-\gamma} = \beta (1+r) \mathbb{E} \Bigl[ \bigl( (1+r) s + y_2 \bigr)^{-\gamma} \Bigr]
    \tag{3.1}
$$
which is a nonlinear equation in a single unknown, $s$.